In [2]:
import torch
import pandas as pd
from datasets import Dataset, load_dataset
from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerModel, TimeSeriesTransformerForPrediction
import numpy as np
import os 

/Users/dorianfitton/opt/anaconda3/envs/bgd/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (5.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
/Users/dorianfitton/opt/anaconda3/envs/bgd/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
if not os.path.exists('/Users/dorianfitton/Library/Mobile Documents/com~apple~CloudDocs/Documents/Cours_Télécom/fil_rouge/fil_rouge.nosync/examples/datasets/EPEX_FR_2_transformer.csv'):

    df = pd.read_csv("/Users/dorianfitton/Library/Mobile Documents/com~apple~CloudDocs/Documents/Cours_Télécom/fil_rouge/fil_rouge.nosync/examples/datasets/EPEX_FR_NEW_UTC.csv")
    df['Date'] = pd.to_datetime(df['Date'])
    # Create features for positional encoding 
    df['Hour of the day'] = df['Date'].dt.hour
    df['Day of the week'] = df['Date'].dt.dayofweek
    df['Day of the year'] = df['Date'].dt.dayofyear
    df['Year'] = df['Date'].dt.year
    df['Month of the year'] = df['Date'].dt.month
    print(df.head())

    df.to_csv("/Users/dorianfitton/Library/Mobile Documents/com~apple~CloudDocs/Documents/Cours_Télécom/fil_rouge/fil_rouge.nosync/examples/datasets/EPEX_FR_2_transformer.csv")
    print("CSV done and saved")

else:
    print("Transformer csv already exists")

Transformer csv already exists


In [4]:
df = pd.read_csv("/Users/dorianfitton/Library/Mobile Documents/com~apple~CloudDocs/Documents/Cours_Télécom/fil_rouge/fil_rouge.nosync/examples/datasets/EPEX_FR_2_transformer.csv")
df['Date'] = pd.to_datetime(df['Date'])
dataset = Dataset.from_pandas(df)
df.head()

,Unnamed: 0,Date,Load forecast,Generation forecast,Price,Hour of the day,Day of the week,Day of the year,Year,Month of the year
0,0,2017-01-01 00:00:00,73650.0,69539.0,58.23,0,6,1,2017,1
1,1,2017-01-01 01:00:00,72350.0,67376.0,51.95,1,6,1,2017,1
2,2,2017-01-01 02:00:00,68750.0,65412.0,47.27,2,6,1,2017,1
3,3,2017-01-01 03:00:00,65900.0,64557.0,45.49,3,6,1,2017,1
4,4,2017-01-01 04:00:00,65000.0,64552.0,44.50,4,6,1,2017,1


In [5]:
device = torch.device("cpu")

In [6]:
# Choix des paramètres 
nb_time_feat = 7
context = 90

In [7]:

# Séparation du dataset en données de test et train (pas nécessaire)
train_dataset = dataset.filter(lambda x: x['Date'].year < 2021)
#test_dataset = dataset.filter(lambda x: x['Date'].date() == pd.to_datetime('2021-01-01').date())

Filter:   0%|          | 0/52584 [00:00<?, ? examples/s]

In [8]:
# On fait la prédiction à partir de la première date 
date = train_dataset['Date'][24*context].date()

# On sépare les données d'entrainement et les données de test (ici on a un jour de test)

past_dataset = train_dataset.filter(lambda x: x['Date'].date() < date)
print("past_dataset shape: ", past_dataset.shape)

future_dataset = train_dataset.filter(lambda x: x['Date'].date() == date)
print("future_dataset shape: ", future_dataset.shape)

Filter:   0%|          | 0/35064 [00:00<?, ? examples/s]

past_dataset shape:  (2160, 10)


Filter:   0%|          | 0/35064 [00:00<?, ? examples/s]

future_dataset shape:  (24, 10)


In [9]:
# On définit quelles vont être les données utilisées/suprimées pour l'entrainement en utilisant un masque (en cas de données manquantes / NaN)
past_observed_mask = torch.ones(24*context).reshape(1, -1)
print("past_observed_mask shape:",past_observed_mask.shape)

past_observed_mask shape: torch.Size([1, 2160])


In [10]:
# On définit la matrice des features (variables exogènes et positional encoding) sur les données d'entrainement et les données de test

past_time_features = torch.tensor(list(zip(past_dataset['Day of the week'],
                                           past_dataset['Hour of the day'],
                                           past_dataset['Day of the year'],
                                           past_dataset['Year'],
                                           past_dataset['Month of the year'],
                                           past_dataset['Load forecast'],
                                           past_dataset['Generation forecast'])))\
                                          .reshape(1, -1, nb_time_feat)

print("past_time_features shape:", past_time_features.shape)

future_time_features = torch.tensor(list(zip(future_dataset['Day of the week'],
                                             future_dataset['Hour of the day'],
                                             future_dataset['Day of the year'],
                                             future_dataset['Year'],
                                             future_dataset['Month of the year'],
                                             future_dataset['Load forecast'],
                                             future_dataset['Generation forecast'])))\
                                             .reshape(1, -1, nb_time_feat)
print("future_time_features shape:",future_time_features.shape)

past_time_features shape: torch.Size([1, 2160, 7])
future_time_features shape: torch.Size([1, 24, 7])


In [11]:
# On définit les valeurs de la variable endogène pour les données d'entrainement et pour les données de test

past_values = torch.tensor(past_dataset['Price']).reshape(1, -1)
print("past_values shape:",past_values.shape)

future_values = torch.tensor(future_dataset['Price']).reshape(1, -1)
print("future_values shape:",future_values.shape)

past_values shape: torch.Size([1, 2160])
future_values shape: torch.Size([1, 24])


In [12]:
# On transfert les matrices vers le gpu 
past_values = past_values.to(device)
past_time_features = past_time_features.to(device)
past_observed_mask = past_observed_mask.to(device)
future_values = future_values.to(device)
future_time_features = future_time_features.to(device)

print("past_time_features shape:", past_time_features.shape)
print("past_values shape:",past_values.shape)
print("past_observed_mask shape:",past_observed_mask.shape)

print("future_time_features shape:",future_time_features.shape)
print("future_values shape:",future_values.shape)

past_time_features shape: torch.Size([1, 2160, 7])
past_values shape: torch.Size([1, 2160])
past_observed_mask shape: torch.Size([1, 2160])
future_time_features shape: torch.Size([1, 24, 7])
future_values shape: torch.Size([1, 24])


In [13]:
# Initializing a Time Series Transformer configuration with 24 time steps for prediction
configuration = TimeSeriesTransformerConfig(prediction_length   = 24, 
                                            context_length      = 24 * (context-1) + 17, 
                                            input_size          = 1, 
                                            output_size         = 1, 
                                            num_time_features   = nb_time_feat )

# Randomly initializing a model (with random weights) from the configuration
model = TimeSeriesTransformerForPrediction(configuration)

# Accessing the model configuration
configuration = model.config
model = model.to(device)



# during training, one provides both past and future values
# as well as possible additional features
outputs = model(
    past_values=past_values,
    past_time_features=past_time_features,
    past_observed_mask=past_observed_mask,
    future_values=future_values,
    future_time_features=future_time_features
)

loss = outputs.loss
loss.backward()

# during inference, one only provides past values
# as well as possible additional features
# the model autoregressively generates future values
outputs = model.generate(
    past_values=past_values,
    past_time_features=past_time_features,
    past_observed_mask=past_observed_mask,
    future_time_features=future_time_features
)


In [14]:
mean_prediction = outputs.sequences.mean(dim=1).detach().cpu()
df_pred = pd.DataFrame(mean_prediction.numpy())
df_pred

#df_pred.to_csv('/Users/dorianfitton/Library/Mobile Documents/com~apple~CloudDocs/Documents/Cours_Télécom/fil_rouge/fil_rouge.nosync/forecasts/transformer_pred.csv')

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,-13.782734,-26.099804,-3.193981,-14.078363,-12.001953,-3.627244,-12.409479,-21.700716,-15.958526,0.644061,...,-1.341909,14.395869,4.388855,-2.381189,-19.363369,3.30722,-28.608604,-3.585922,5.668192,0.359775


In [15]:
model.num_parameters()

399043

In [16]:
model

TimeSeriesTransformerForPrediction(
  (model): TimeSeriesTransformerModel(
    (scaler): TimeSeriesMeanScaler()
    (encoder): TimeSeriesTransformerEncoder(
      (value_embedding): TimeSeriesValueEmbedding(
        (value_projection): Linear(in_features=16, out_features=64, bias=False)
      )
      (embed_positions): TimeSeriesSinusoidalPositionalEmbedding(2177, 64)
      (layers): ModuleList(
        (0-1): 2 x TimeSeriesTransformerEncoderLayer(
          (self_attn): TimeSeriesTransformerAttention(
            (k_proj): Linear(in_features=64, out_features=64, bias=True)
            (v_proj): Linear(in_features=64, out_features=64, bias=True)
            (q_proj): Linear(in_features=64, out_features=64, bias=True)
            (out_proj): Linear(in_features=64, out_features=64, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=64, out_features=32

In [23]:
from huggingface_hub import hf_hub_download
import torch
from transformers import TimeSeriesTransformerForPrediction

file = hf_hub_download(
    repo_id="kashif/tourism-monthly-batch", filename="train-batch.pt", repo_type="dataset"
)
batch = torch.load(file)

model = TimeSeriesTransformerForPrediction.from_pretrained(
    "huggingface/time-series-transformer-tourism-monthly"
)

# during training, one provides both past and future values
# as well as possible additional features
outputs = model(
    past_values=batch["past_values"],
    past_time_features=batch["past_time_features"],
    past_observed_mask=batch["past_observed_mask"],
    static_categorical_features=batch["static_categorical_features"],
    static_real_features=batch["static_real_features"],
    future_values=batch["future_values"],
    future_time_features=batch["future_time_features"],
)

loss = outputs.loss
loss.backward()

# during inference, one only provides past values
# as well as possible additional features
# the model autoregressively generates future values
outputs = model.generate(
    past_values=batch["past_values"],
    past_time_features=batch["past_time_features"],
    past_observed_mask=batch["past_observed_mask"],
    static_categorical_features=batch["static_categorical_features"],
    static_real_features=batch["static_real_features"],
    future_time_features=batch["future_time_features"],
)

mean_prediction = outputs.sequences.mean(dim=1)

In [ ]:
batch['past_time_features'].shape

torch.Size([64, 61, 2])